In [1]:
import pandas as pd

In [2]:
productmaster = pd.read_excel("Product Master.xlsx")
productmaster

,MATERIAL,MATERIAL REF,DESCRIPTION
0,jeanpantl,jeanpantl,Jean Pant Size L
1,jeanpantlf,jeanpantl,Jean Pant Size L FREE
2,jeanpantm,jeanpantm,Jean Pant Size M
3,jeanpantmf,jeanpantm,Jean Pant Size M FREE
4,jeanpants,jeanpants,Jean Pant Size S
5,jeanpants,jeanpants,Jean Pant Size S FREE
6,slimfitpantl,slimfitpantl,Slim Fit Pant Size L
7,slimfitpantlf,slimfitpantl,Slim Fit Pant Size L FREE
8,slimfitpantm,slimfitpantm,Slim Fit Pant Size M
9,slimfitpantmf,slimfitpantm,Slim Fit Pant Size M FREE


In [3]:
rawdata = pd.read_excel("orders.xlsx")
rawdata

,ORDER_NO,ORDER_CDATE,MATERIAL,QUANTITY
0,ORD4198,2025-01-29,tshirtwhl,1
1,ORD4198,2025-01-29,jeanpantl,2
2,ORD5591,2025-01-24,jeanpantl,1
3,ORD6754,2025-01-27,jeanpants,3
4,ORD6754,2025-01-27,jeanpantl,5
...,...,...,...,...
29663,ORD7138,2025-01-14,jeanpants,1
29664,ORD5150,2025-02-02,jeanpantl,4
29665,ORD5150,2025-02-02,jeanpants,5
29666,ORD5150,2025-02-02,jeanpantm,5


In [4]:
#Merge and Clean Raw Data
mergedata = rawdata.merge(productmaster[['MATERIAL', 'MATERIAL REF']], how='left', on='MATERIAL')
mergedata = mergedata[['MATERIAL', 'ORDER_CDATE', 'ORDER_NO', 'MATERIAL REF', 'QUANTITY']]
mergedata['MAT'] = mergedata.apply(lambda d: d['MATERIAL REF'] if pd.notna(d['MATERIAL REF']) else d['MATERIAL'], axis=1)

mergedata = mergedata.merge(productmaster[['MATERIAL','DESCRIPTION']], how='left', left_on='MAT',right_on='MATERIAL')
mergedata = mergedata.groupby(['ORDER_NO','MAT','DESCRIPTION']).agg(QUANTITY = ('QUANTITY','sum')).reset_index()

mergedata['PRODUCT'] = mergedata['DESCRIPTION'] + ' (' + mergedata['MAT'] +')_ [' + mergedata['QUANTITY'].astype(int).astype(str) + ']'
mergedata = mergedata[['ORDER_NO', 'PRODUCT']]
mergedata[['ORDER_NO', 'PRODUCT']] = mergedata[['ORDER_NO', 'PRODUCT']].astype(str)
mergedata

,ORDER_NO,PRODUCT
0,ORD1000,Jean Pant Size L (jeanpantl)_ [2]
1,ORD1000,Jean Pant Size M (jeanpantm)_ [3]
2,ORD1000,Jean Pant Size S (jeanpants)_ [4]
3,ORD1000,Jean Pant Size S FREE (jeanpants)_ [4]
4,ORD1000,Slim Fit Pant Size M (slimfitpantm)_ [3]
...,...,...
26091,ORD9999,Jean Pant Size M (jeanpantm)_ [4]
26092,ORD9999,Jean Pant Size S (jeanpants)_ [2]
26093,ORD9999,Jean Pant Size S FREE (jeanpants)_ [2]
26094,ORD9999,Slim Fit Pant Size M (slimfitpantm)_ [2]


In [5]:
groupdata = mergedata.groupby('ORDER_NO')['PRODUCT'].agg(lambda x: ','.join(x)).reset_index()
groupdata

,ORDER_NO,PRODUCT
0,ORD1000,"Jean Pant Size L (jeanpantl)_ [2],Jean Pant Si..."
1,ORD1001,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si..."
2,ORD1002,"Jean Pant Size M (jeanpantm)_ [1],T-shirt Whit..."
3,ORD1003,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si..."
4,ORD1004,"Jean Pant Size L (jeanpantl)_ [1],Jean Pant Si..."
...,...,...
6050,ORD9993,"Jean Pant Size L (jeanpantl)_ [2],Jean Pant Si..."
6051,ORD9995,"Jean Pant Size M (jeanpantm)_ [3],Jean Pant Si..."
6052,ORD9996,"Jean Pant Size M (jeanpantm)_ [2],Jean Pant Si..."
6053,ORD9997,"Jean Pant Size L (jeanpantl)_ [2],Jean Pant Si..."


In [6]:
mixdata = groupdata.groupby(['PRODUCT']).count().reset_index()
mixdata.sort_values(by=['ORDER_NO'],ascending=False,inplace=True)
mixdata.columns = ['shortname (sku) [qty]','count order']
mixdata

,shortname (sku) [qty],count order
4377,Slim Fit Pant Size M (slimfitpantm)_ [1],33
4415,T-shirt White Size L (tshirtwhl)_ [2],32
4058,Jean Pant Size M (jeanpantm)_ [5],31
4417,T-shirt White Size L (tshirtwhl)_ [4],30
4384,Slim Fit Pant Size M (slimfitpantm)_ [2],30
...,...,...
1560,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si...",1
1561,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si...",1
1562,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si...",1
1564,"Jean Pant Size L (jeanpantl)_ [3],Jean Pant Si...",1


In [7]:
split_df = mixdata[:1000]['shortname (sku) [qty]'].str.split(',', expand=True)
result = pd.concat([mixdata[:1000][['count order']], split_df], axis=1)
result.columns = ['count order'] + [f'product [qty] {i+1}' for i in range(split_df.shape[1])]
result.reset_index(drop=True,inplace=True)
result

,count order,product [qty] 1,product [qty] 2,product [qty] 3,product [qty] 4,product [qty] 5,product [qty] 6
0,33,Slim Fit Pant Size M (slimfitpantm)_ [1],None,None,None,None,None
1,32,T-shirt White Size L (tshirtwhl)_ [2],None,None,None,None,None
2,31,Jean Pant Size M (jeanpantm)_ [5],None,None,None,None,None
3,30,T-shirt White Size L (tshirtwhl)_ [4],None,None,None,None,None
4,30,Slim Fit Pant Size M (slimfitpantm)_ [2],None,None,None,None,None
...,...,...,...,...,...,...,...
995,1,Jean Pant Size L (jeanpantl)_ [5],Jean Pant Size M (jeanpantm)_ [3],Jean Pant Size S (jeanpants)_ [8],Jean Pant Size S FREE (jeanpants)_ [8],Slim Fit Pant Size M (slimfitpantm)_ [3],T-shirt White Size L (tshirtwhl)_ [3]
996,1,Jean Pant Size L (jeanpantl)_ [5],Jean Pant Size M (jeanpantm)_ [3],Jean Pant Size S (jeanpants)_ [8],Jean Pant Size S FREE (jeanpants)_ [8],Slim Fit Pant Size M (slimfitpantm)_ [3],T-shirt White Size L (tshirtwhl)_ [4]
997,1,Jean Pant Size L (jeanpantl)_ [5],Jean Pant Size M (jeanpantm)_ [3],Jean Pant Size S (jeanpants)_ [8],Jean Pant Size S FREE (jeanpants)_ [8],Slim Fit Pant Size M (slimfitpantm)_ [3],T-shirt White Size L (tshirtwhl)_ [5]
998,1,Jean Pant Size L (jeanpantl)_ [5],Jean Pant Size M (jeanpantm)_ [3],Jean Pant Size S (jeanpants)_ [8],Jean Pant Size S FREE (jeanpants)_ [8],Slim Fit Pant Size M (slimfitpantm)_ [4],T-shirt White Size L (tshirtwhl)_ [1]
